In [5]:
import torch
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import sys
sys.path.append('../..')
from utils.utils import samples_per_class

# var size

In [6]:
var_size_path = 'comparison_var_size_results'

# set class groups
spc = samples_per_class(torch.load('/mnt/dtafler/test/timm/vit_base_patch14_dinov2.lvd142m/cifar100_0.01/train/labels.pt'))
few_shot = spc <= 20
med_shot = (100 >= spc) & (spc > 20)
many_shot = spc > 100
# class_groups = [many_shot, med_shot, few_shot]

# fig, axs = plt.subplots(2, 3, figsize=(15, 10), sharey=True, sharex=True)

# make dataframe
index = pd.MultiIndex.from_product([['small', 'large'], ['var_1', 'var_2', 'var_3']], names=['size', 'var'])
columns = ['overall', 'many', 'medium', 'few']
df = pd.DataFrame(index=index, columns=columns)

for row, size in enumerate(['small', 'large']):
    for col, var in enumerate(['var_1', 'var_2', 'var_3']):
        accs = []
        per_class_accs = []
        
        for run in range(5):
            accs.append(torch.load(f'{var_size_path}/{size}/{var}/run_{run}/acc.pt'))
            per_class_accs.append(torch.load(f'{var_size_path}/{size}/{var}/run_{run}/per_class.pt'))
        
        accs = torch.stack(accs, dim=0)
        per_class_accs = torch.stack(per_class_accs, dim=0)
        
        avg_acc = accs.mean(dim=0)
        avg_per_class_acc = per_class_accs.mean(dim=0)
        
        df.loc[(size, var), 'overall'] = avg_acc.item()
        df.loc[(size, var), 'many'] = avg_per_class_acc[many_shot].mean().item()
        df.loc[(size, var), 'medium'] = avg_per_class_acc[med_shot].mean().item()
        df.loc[(size, var), 'few'] = avg_per_class_acc[few_shot].mean().item()
        
        
# same for small vars
var_size_path = 'comparison_var_size_results_small_var_cifar100_0.01'

# make dataframe
index = pd.MultiIndex.from_product([['small', 'large'], ['var_0.25', 'var_0.5', 'var_0.75']], names=['size', 'var'])
columns = ['overall', 'many', 'medium', 'few']
df_small = pd.DataFrame(index=index, columns=columns)

for row, size in enumerate(['small', 'large']):
    for col, var in enumerate(['var_0.25', 'var_0.5', 'var_0.75']):
        accs = []
        per_class_accs = []
        
        for run in range(5):
            accs.append(torch.load(f'{var_size_path}/{size}/{var}/run_{run}/acc.pt'))
            per_class_accs.append(torch.load(f'{var_size_path}/{size}/{var}/run_{run}/per_class.pt'))
        
        accs = torch.stack(accs, dim=0)
        per_class_accs = torch.stack(per_class_accs, dim=0)
        
        avg_acc = accs.mean(dim=0)
        avg_per_class_acc = per_class_accs.mean(dim=0)
        
        df_small.loc[(size, var), 'overall'] = avg_acc.item()
        df_small.loc[(size, var), 'many'] = avg_per_class_acc[many_shot].mean().item()
        df_small.loc[(size, var), 'medium'] = avg_per_class_acc[med_shot].mean().item()
        df_small.loc[(size, var), 'few'] = avg_per_class_acc[few_shot].mean().item()
        
        
df = df * 100
df = df.map(lambda x: round(x, 2))
df = df.map(lambda x: f"{x:.2f}") # so latex doesn't add zeros
df = df.rename(index={'var_1': '1', 'var_2': '2', 'var_3': '3'})
df = df.rename_axis(index={'size': 'CVAE'})

       
df_small = df_small * 100
df_small = df_small.map(lambda x: round(x, 2))
df_small = df_small.map(lambda x: f"{x:.2f}") # so latex doesn't add zeros
df_small = df_small.rename(index={'var_0.25': '0.25', 'var_0.5': '0.5', 'var_0.75': '0.75'})
df_small = df_small.rename_axis(index={'size': 'CVAE'})
       
result_df = pd.concat([df, df_small]).sort_index()
# print(result_df.index)

ordered_idx = sorted(result_df.index, key=lambda x: ('small' if x[0] == 'large' else 'large', x[1]))
var_size_100_df = result_df.reindex(ordered_idx)

var_size_100_df

overall   many medium    few
CVAE  var                              
small 0.25   83.64  93.09  85.02  71.47
      0.5    83.91  93.31  84.85  72.28
      0.75   84.42  93.20  85.22  73.63
      1      84.22  93.37  85.12  72.89
      2      84.02  93.31  84.74  72.74
      3      84.19  93.14  84.42  73.83
large 0.25   83.67  93.17  84.69  71.83
      0.5    84.83  92.87  85.32  75.21
      0.75   85.67  92.37  85.43  78.37
      1      84.62  91.95  84.89  76.03
      2      83.26  92.19  84.02  72.33
      3      83.70  92.74  84.60  72.50

In [7]:
var_size_100_df = var_size_100_df.reset_index()

# Separate the small and large sections
small_df = var_size_100_df[var_size_100_df['CVAE'] == 'small']
large_df = var_size_100_df[var_size_100_df['CVAE'] == 'large']

# Find the max values for each column except 'CVAE' and 'var'
max_values_small = small_df.loc[:, 'overall':].max()
max_values_large = large_df.loc[:, 'overall':].max()

# Apply the bold formatting
def highlight_max(df, max_values):
    for col in df.columns[2:]:  # Skip 'CVAE' and 'var' columns
        df[col] = df[col].apply(lambda x: f"\\textbf{{{x}}}" if x == max_values[col] else x)
    return df

small_df = highlight_max(small_df, max_values_small)
large_df = highlight_max(large_df, max_values_large)

# Combine the small and large sections back together
highlighted_df = pd.concat([small_df, large_df])

# Set the index back to 'CVAE' and 'var'
highlighted_df = highlighted_df.set_index(['CVAE', 'var'])
highlighted_df


#### reorder ####
highlighted_df = highlighted_df.reset_index()

# Separate the small and large sections
small_df = highlighted_df[highlighted_df['CVAE'] == 'small']
large_df = highlighted_df[highlighted_df['CVAE'] == 'large']

columns = pd.MultiIndex.from_product([['Small', 'Large'], ['overall', 'many', 'medium', 'few']])
final_df = pd.DataFrame(columns=columns)


small_df.set_index('var').drop(columns='CVAE')

final_df['Small'] = small_df.set_index('var').drop(columns='CVAE')
final_df['Large'] = large_df.set_index('var').drop(columns='CVAE')
final_df.index = ['0.25', '0.5', '0.75', '1', '2', '3']

/tmp/ipykernel_4106547/2601336002.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(lambda x: f"\\textbf{{{x}}}" if x == max_values[col] else x)


In [8]:
caption = 'Mean accuracies for \\textbf{Small} and \\textbf{Large} CVAE architectures and sampling variances \(\\boldsymbol{\sigma^2}\) on CIFAR100 LT with \(\\rho = 100\), averaged over 5 runs. \\textbf{O}: Overall, \\textbf{MA}: Many-shot classes, \\textbf{ME}: Medium-shot classes, \\textbf{F}: Few-shot classes.'
label = 'tab:var_size_100'       
 
latex_table = final_df.to_latex(
    column_format='lcccccccc',
    escape=False,
    multicolumn=True,
    multirow=True
)

latex_table = f"\\begin{{table}}[!ht]\n\\centering\n{latex_table}\n\\caption{{{caption}}}\n\\label{{{label}}}\n\\end{{table}}"
latex_table = latex_table.replace('& \multicolumn{4}{r}{Small} & \multicolumn{4}{r}{Large} \\\\', '& \multicolumn{4}{c}{\\textbf{Small}} & \multicolumn{4}{c}{\\textbf{Large}} \\\\\n\cmidrule(lr){2-5} \cmidrule(lr){6-9}')
latex_table = latex_table.replace('& overall & many & medium & few & overall & many & medium & few \\', '\\(\\boldsymbol{\sigma^2}\) & \\textbf{O} & \\textbf{MA} & \\textbf{ME} & \\textbf{F} & \\textbf{O} & \\textbf{MA} & \\textbf{ME} & \\textbf{F} \\')
print(latex_table)

\begin{table}[!ht]
\centering
\begin{tabular}{lcccccccc}
\toprule
 & \multicolumn{4}{c}{\textbf{Small}} & \multicolumn{4}{c}{\textbf{Large}} \\
\cmidrule(lr){2-5} \cmidrule(lr){6-9}
 \(\boldsymbol{\sigma^2}\) & \textbf{O} & \textbf{MA} & \textbf{ME} & \textbf{F} & \textbf{O} & \textbf{MA} & \textbf{ME} & \textbf{F} \\
\midrule
0.25 & 83.64 & 93.09 & 85.02 & 71.47 & 83.67 & \textbf{93.17} & 84.69 & 71.83 \\
0.5 & 83.91 & 93.31 & 84.85 & 72.28 & 84.83 & 92.87 & 85.32 & 75.21 \\
0.75 & \textbf{84.42} & 93.20 & \textbf{85.22} & 73.63 & \textbf{85.67} & 92.37 & \textbf{85.43} & \textbf{78.37} \\
1 & 84.22 & \textbf{93.37} & 85.12 & 72.89 & 84.62 & 91.95 & 84.89 & 76.03 \\
2 & 84.02 & 93.31 & 84.74 & 72.74 & 83.26 & 92.19 & 84.02 & 72.33 \\
3 & 84.19 & 93.14 & 84.42 & \textbf{73.83} & 83.70 & 92.74 & 84.60 & 72.50 \\
\bottomrule
\end{tabular}

\caption{Mean accuracies for \textbf{Small} and \textbf{Large} CVAE architectures and sampling variances \(\boldsymbol{\sigma^2}\) on CIFAR100-LT with

<>:1: SyntaxWarning: invalid escape sequence '\('
<>:12: SyntaxWarning: invalid escape sequence '\m'
<>:12: SyntaxWarning: invalid escape sequence '\m'
<>:13: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\('
<>:12: SyntaxWarning: invalid escape sequence '\m'
<>:12: SyntaxWarning: invalid escape sequence '\m'
<>:13: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_4106547/917071495.py:1: SyntaxWarning: invalid escape sequence '\('
  caption = 'Mean accuracies for \\textbf{Small} and \\textbf{Large} CVAE architectures and sampling variances \(\\boldsymbol{\sigma^2}\) on CIFAR100-LT with \(\\rho = 100\), averaged over 5 runs. \\textbf{O}: Overall, \\textbf{MA}: Many-shot classes, \\textbf{ME}: Medium-shot classes, \\textbf{F}: Few-shot classes.'
/tmp/ipykernel_4106547/917071495.py:12: SyntaxWarning: invalid escape sequence '\m'
  latex_table = latex_table.replace('& \multicolumn{4}{r}{Small} & \multicolumn{4}{r}{Large} \\\\', '& \mult

## cifar 10

In [9]:
var_size_path = 'comparison_var_size_results_cifar10'

# set class groups
spc = samples_per_class(torch.load('/mnt/dtafler/test/timm/vit_base_patch14_dinov2.lvd142m/cifar10_0.01/train/labels.pt'))
few_shot = spc <= 20
med_shot = (100 >= spc) & (spc > 20)
many_shot = spc > 100

# make dataframe
index = pd.MultiIndex.from_product([['small', 'large'], ['var_1', 'var_2', 'var_3']], names=['size', 'var'])
columns = ['overall', 'many', 'medium', 'few']
df = pd.DataFrame(index=index, columns=columns)

for row, size in enumerate(['small', 'large']):
    for col, var in enumerate(['var_1', 'var_2', 'var_3']):
        accs = []
        per_class_accs = []
        
        for run in range(5):
            accs.append(torch.load(f'{var_size_path}/{size}/{var}/run_{run}/acc.pt'))
            per_class_accs.append(torch.load(f'{var_size_path}/{size}/{var}/run_{run}/per_class.pt'))
        
        accs = torch.stack(accs, dim=0)
        per_class_accs = torch.stack(per_class_accs, dim=0)
        
        avg_acc = accs.mean(dim=0)
        avg_per_class_acc = per_class_accs.mean(dim=0)
        
        df.loc[(size, var), 'overall'] = avg_acc.item()
        df.loc[(size, var), 'many'] = avg_per_class_acc[many_shot].mean().item()
        df.loc[(size, var), 'medium'] = avg_per_class_acc[med_shot].mean().item()
        df.loc[(size, var), 'few'] = avg_per_class_acc[few_shot].mean().item()
        

var_size_path = 'comparison_var_size_results_small_var_cifar10_0.01'


# make dataframe
index = pd.MultiIndex.from_product([['small', 'large'], ['var_0.25', 'var_0.5', 'var_0.75']], names=['size', 'var'])
columns = ['overall', 'many', 'medium', 'few']
df_small = pd.DataFrame(index=index, columns=columns)

for row, size in enumerate(['small', 'large']):
    for col, var in enumerate(['var_0.25', 'var_0.5', 'var_0.75']):
        accs = []
        per_class_accs = []
        
        for run in range(5):
            accs.append(torch.load(f'{var_size_path}/{size}/{var}/run_{run}/acc.pt'))
            per_class_accs.append(torch.load(f'{var_size_path}/{size}/{var}/run_{run}/per_class.pt'))
        
        accs = torch.stack(accs, dim=0)
        per_class_accs = torch.stack(per_class_accs, dim=0)
        
        avg_acc = accs.mean(dim=0)
        avg_per_class_acc = per_class_accs.mean(dim=0)
        
        df_small.loc[(size, var), 'overall'] = avg_acc.item()
        df_small.loc[(size, var), 'many'] = avg_per_class_acc[many_shot].mean().item()
        df_small.loc[(size, var), 'medium'] = avg_per_class_acc[med_shot].mean().item()
        df_small.loc[(size, var), 'few'] = avg_per_class_acc[few_shot].mean().item()
        
df = df * 100
df = df.map(lambda x: round(x, 2))
df = df.map(lambda x: f"{x:.2f}") # so latex doesn't add zeros
df = df.rename(index={'var_1': '1', 'var_2': '2', 'var_3': '3'})
df = df.rename_axis(index={'size': 'CVAE'})
        
df_small = df_small * 100
df_small = df_small.map(lambda x: round(x, 2))
df_small = df_small.map(lambda x: f"{x:.2f}") # so latex doesn't add zeros
df_small = df_small.rename(index={'var_0.25': '0.25', 'var_0.5': '0.5', 'var_0.75': '0.75'})
df_small = df_small.rename_axis(index={'size': 'CVAE'})
       
result_df = pd.concat([df, df_small]).sort_index()

ordered_idx = sorted(result_df.index, key=lambda x: ('small' if x[0] == 'large' else 'large', x[1]))
var_size_10_df = result_df.reindex(ordered_idx)
var_size_10_df = var_size_10_df.drop(columns=['few'])
var_size_10_df

overall   many medium
CVAE  var                       
small 0.25   97.61  98.37  94.58
      0.5    97.72  98.38  95.05
      0.75   97.77  98.46  95.02
      1      97.72  98.39  95.05
      2      97.58  98.32  94.62
      3      97.68  98.34  95.01
large 0.25   97.64  98.36  94.75
      0.5    97.80  98.37  95.52
      0.75   98.05  98.42  96.60
      1      98.11  98.35  97.19
      2      97.81  98.21  96.18
      3      97.73  98.25  95.65

In [10]:
df = var_size_10_df.map(lambda x: float(x))
print(df.loc[('small'),:].max() - df.loc[('small'),:].min())
print(df.loc[( 'large'),:].max() - df.loc[( 'large'),:].min())

overall    0.19
many       0.14
medium     0.47
dtype: float64
overall    0.47
many       0.21
medium     2.44
dtype: float64


In [11]:
var_size_10_df = var_size_10_df.reset_index()

# Separate the small and large sections
small_df = var_size_10_df[var_size_10_df['CVAE'] == 'small']
large_df = var_size_10_df[var_size_10_df['CVAE'] == 'large']

# Find the max values for each column except 'CVAE' and 'var'
max_values_small = small_df.loc[:, 'overall':].max()
max_values_large = large_df.loc[:, 'overall':].max()

# Apply the bold formatting
def highlight_max(df, max_values):
    for col in df.columns[2:]:  # Skip 'CVAE' and 'var' columns
        df[col] = df[col].apply(lambda x: f"\\textbf{{{x}}}" if x == max_values[col] else x)
    return df

small_df = highlight_max(small_df, max_values_small)
large_df = highlight_max(large_df, max_values_large)

# Combine the small and large sections back together
highlighted_df = pd.concat([small_df, large_df])

# Set the index back to 'CVAE' and 'var'
highlighted_df = highlighted_df.set_index(['CVAE', 'var'])
highlighted_df


#### reorder ####
highlighted_df = highlighted_df.reset_index()

# Separate the small and large sections
small_df = highlighted_df[highlighted_df['CVAE'] == 'small']
large_df = highlighted_df[highlighted_df['CVAE'] == 'large']

columns = pd.MultiIndex.from_product([['Small', 'Large'], ['overall', 'many', 'medium']])
final_df = pd.DataFrame(columns=columns)


small_df.set_index('var').drop(columns='CVAE')

final_df['Small'] = small_df.set_index('var').drop(columns='CVAE')
final_df['Large'] = large_df.set_index('var').drop(columns='CVAE')
final_df.index = ['0.25', '0.5', '0.75', '1', '2', '3']
final_df

/tmp/ipykernel_4106547/434120024.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(lambda x: f"\\textbf{{{x}}}" if x == max_values[col] else x)


Small                                           Large  \
             overall            many          medium         overall   
0.25           97.61           98.37           94.58           97.64   
0.5            97.72           98.38  \textbf{95.05}           97.80   
0.75  \textbf{97.77}  \textbf{98.46}           95.02           98.05   
1              97.72           98.39  \textbf{95.05}  \textbf{98.11}   
2              97.58           98.32           94.62           97.81   
3              97.68           98.34           95.01           97.73   

                                      
                many          medium  
0.25           98.36           94.75  
0.5            98.37           95.52  
0.75  \textbf{98.42}           96.60  
1              98.35  \textbf{97.19}  
2              98.21           96.18  
3              98.25           95.65

In [12]:
caption = 'Mean accuracies for \\textbf{Small} and \\textbf{Large} CVAE architectures and sampling variances \(\\boldsymbol{\sigma^2}\) on CIFAR10 LT with \(\\rho = 100\), averaged over 5 runs. \\textbf{O}: Overall, \\textbf{MA}: Many-shot classes, \\textbf{ME}: Medium-shot classes, \\textbf{F}: Few-shot classes.'
label = 'tab:var_size_10'       
 
latex_table = final_df.to_latex(
    column_format='lcccccc',
    escape=False,
    multicolumn=True,
    multirow=True
)

latex_table = f"\\begin{{table}}[!ht]\n\\centering\n{latex_table}\n\\caption{{{caption}}}\n\\label{{{label}}}\n\\end{{table}}"


latex_table = latex_table.replace('& \multicolumn{3}{r}{Small} & \multicolumn{3}{r}{Large} \\\\', '& \multicolumn{3}{c}{\\textbf{Small}} & \multicolumn{3}{c}{\\textbf{Large}} \\\\\n\cmidrule(lr){2-4} \cmidrule(lr){5-7}')
latex_table = latex_table.replace('& overall & many & medium & overall & many & medium \\\\', '\\(\\boldsymbol{\sigma^2}\) & \\textbf{O} & \\textbf{MA} & \\textbf{ME} & \\textbf{O} & \\textbf{MA} & \\textbf{ME} \\\\')

print(latex_table)

\begin{table}[!ht]
\centering
\begin{tabular}{lcccccc}
\toprule
 & \multicolumn{3}{c}{\textbf{Small}} & \multicolumn{3}{c}{\textbf{Large}} \\
\cmidrule(lr){2-4} \cmidrule(lr){5-7}
 \(\boldsymbol{\sigma^2}\) & \textbf{O} & \textbf{MA} & \textbf{ME} & \textbf{O} & \textbf{MA} & \textbf{ME} \\
\midrule
0.25 & 97.61 & 98.37 & 94.58 & 97.64 & 98.36 & 94.75 \\
0.5 & 97.72 & 98.38 & \textbf{95.05} & 97.80 & 98.37 & 95.52 \\
0.75 & \textbf{97.77} & \textbf{98.46} & 95.02 & 98.05 & \textbf{98.42} & 96.60 \\
1 & 97.72 & 98.39 & \textbf{95.05} & \textbf{98.11} & 98.35 & \textbf{97.19} \\
2 & 97.58 & 98.32 & 94.62 & 97.81 & 98.21 & 96.18 \\
3 & 97.68 & 98.34 & 95.01 & 97.73 & 98.25 & 95.65 \\
\bottomrule
\end{tabular}

\caption{Mean accuracies for \textbf{Small} and \textbf{Large} CVAE architectures and sampling variances \(\boldsymbol{\sigma^2}\) on CIFAR10-LT with \(\rho = 100\), averaged over 5 runs. \textbf{O}: Overall, \textbf{MA}: Many-shot classes, \textbf{ME}: Medium-shot classes, \textbf{

<>:1: SyntaxWarning: invalid escape sequence '\('
<>:14: SyntaxWarning: invalid escape sequence '\m'
<>:14: SyntaxWarning: invalid escape sequence '\m'
<>:15: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\('
<>:14: SyntaxWarning: invalid escape sequence '\m'
<>:14: SyntaxWarning: invalid escape sequence '\m'
<>:15: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_4106547/323032484.py:1: SyntaxWarning: invalid escape sequence '\('
  caption = 'Mean accuracies for \\textbf{Small} and \\textbf{Large} CVAE architectures and sampling variances \(\\boldsymbol{\sigma^2}\) on CIFAR10-LT with \(\\rho = 100\), averaged over 5 runs. \\textbf{O}: Overall, \\textbf{MA}: Many-shot classes, \\textbf{ME}: Medium-shot classes, \\textbf{F}: Few-shot classes.'
/tmp/ipykernel_4106547/323032484.py:14: SyntaxWarning: invalid escape sequence '\m'
  latex_table = latex_table.replace('& \multicolumn{3}{r}{Small} & \multicolumn{3}{r}{Large} \\\\', '& \multi

# augmentation

In [9]:
aug_path = 'comparison_augmentation_results'

# set class groups
def get_class_groups_idx(dataset):
    spc = samples_per_class(torch.load(f'/mnt/dtafler/test/timm/vit_base_patch14_dinov2.lvd142m/{dataset}/train/labels.pt'))
    few_shot = spc <= 20
    med_shot = (100 >= spc) & (spc > 20)
    many_shot = spc > 100
    return {'few': few_shot, 'medium': med_shot, 'many': many_shot}

# rows and columns
methods = ['baseline', 'cbs', 'smote', 'adasyn', 'remix', 'ours', 'ours_remix']
datasets = ['cifar100_0.01', 'cifar10_0.01']
metrics = ['overall', 'many', 'medium', 'few']
columns = pd.MultiIndex.from_product([datasets, metrics], names=['dataset', 'metric'])

# make dataframe
df = pd.DataFrame(index=methods, columns=columns)
df.index.name = 'method'

for method in methods:
    
    for dataset in datasets:

            accs = []
            per_class_accs = []
            
            for run in range(5):
                res_path = f'{aug_path}/{dataset}/{method}/run_{run}'
                accs.append(torch.load(f'{res_path}/acc.pt'))
                per_class_accs.append(torch.load(f'{res_path}/per_class.pt'))
            
            accs = torch.stack(accs, dim=0)
            per_class_accs = torch.stack(per_class_accs, dim=0)
            
            avg_acc = accs.mean(dim=0)
            avg_per_class_acc = per_class_accs.mean(dim=0)
            
            df.loc[method, (dataset, 'overall')] = avg_acc.item()
            for metric in ['few', 'medium', 'many']:
                df.loc[method, (dataset, metric)] = avg_per_class_acc[get_class_groups_idx(dataset)[metric]].mean().item()
            
df = df * 100
df = df.map(lambda x: round(x, 2))
df = df.map(lambda x: f"{x:.2f}")
df.columns = pd.MultiIndex.from_product([['CIFAR100 LT', 'CIFAR10 LT'], metrics])
df = df.drop(columns=[('CIFAR10 LT', 'few')])
df.index = ['Baseline', 'CBS', 'SMOTE', 'ADASYN', 'Remix', 'Ours', 'Ours + Remix']
df


CIFAR100 LT                      CIFAR10 LT              
                 overall   many medium    few    overall   many medium
Baseline           80.47  93.80  82.30  63.40      96.88  98.18  91.67
CBS                83.74  93.02  85.18  71.70      97.78  98.43  95.19
SMOTE              83.31  93.42  84.10  71.04      97.69  98.39  94.87
ADASYN             82.87  93.33  83.78  70.07      97.51  98.35  94.14
Remix              84.91  92.65  85.60  75.42      98.08  98.51  96.35
Ours               85.52  91.61  85.54  78.62      98.02  98.29  96.93
Ours + Remix       85.84  90.73  85.61  80.57      98.15  98.10  98.38

In [10]:
for col in df.columns:
    # df[col] = df[col].apply(lambda x: f"\\textbf{{{x}}}" if x == df[col].max() else x)
    max = df[col].max()
    second_max = df[col].sort_values(ascending=False).iloc[1]
    df[col] = df[col].apply(lambda x: f"\\textbf{{{x}}}" if x == max else f"\\underline{{{x}}}" if x == second_max else x)
    
df

CIFAR100 LT                                        \
                        overall               many             medium   
Baseline                  80.47     \textbf{93.80}              82.30   
CBS                       83.74              93.02              85.18   
SMOTE                     83.31  \underline{93.42}              84.10   
ADASYN                    82.87              93.33              83.78   
Remix                     84.91              92.65  \underline{85.60}   
Ours          \underline{85.52}              91.61              85.54   
Ours + Remix     \textbf{85.84}              90.73     \textbf{85.61}   

                                        CIFAR10 LT                     \
                            few            overall               many   
Baseline                  63.40              96.88              98.18   
CBS                       71.70              97.78  \underline{98.43}   
SMOTE                     71.04              97.69              98.39   
ADASYN                    70.07              97.51              98.35   
Remix                     75.42  \underline{98.08}     \textbf{98.51}   
Ours          \underline{78.62}              98.02              98.29   
Ours + Remix     \textbf{80.57}     \textbf{98.15}              98.10   

                                 
                         medium  
Baseline                  91.67  
CBS                       95.19  
SMOTE                     94.87  
ADASYN                    94.14  
Remix                     96.35  
Ours          \underline{96.93}  
Ours + Remix     \textbf{98.38}

In [11]:
caption = 'Mean accuracies for different resampling and data augmentation techniques on CIFAR100 LT and CIFAR10 LT with \(\\rho = 100\), averaged over 5 runs. Best and second best results per column are in bold and underlined, respectively. \\textbf{O}: Overall, \\textbf{MA}: Many-shot classes, \\textbf{ME}: Medium-shot classes, \\textbf{F}: Few-shot classes.'
label = 'tab:aug'       
 
latex_table = df.to_latex(
    column_format='lccccccc',
    escape=False,
    multicolumn=True,
    multirow=False
)

latex_table = f"\\begin{{table}}[!ht]\n\\centering\n{latex_table}\n\\caption{{{caption}}}\n\\label{{{label}}}\n\\end{{table}}"
latex_table = latex_table.replace('& \multicolumn{4}{r}{CIFAR100 LT} & \multicolumn{3}{r}{CIFAR10 LT} \\\\', '& \multicolumn{4}{c}{\\textbf{CIFAR100 LT}} & \multicolumn{3}{c}{\\textbf{CIFAR10 LT}} \\\\\n\cmidrule(lr){2-5} \cmidrule(lr){6-8}')
latex_table = latex_table.replace('& overall & many & medium & few & overall & many & medium \\', '& \\textbf{O} & \\textbf{MA} & \\textbf{ME} & \\textbf{F} & \\textbf{O} & \\textbf{MA} & \\textbf{ME} \\')
print(latex_table)

\begin{table}[!ht]
\centering
\begin{tabular}{lccccccc}
\toprule
 & \multicolumn{4}{c}{\textbf{CIFAR100 LT}} & \multicolumn{3}{c}{\textbf{CIFAR10 LT}} \\
\cmidrule(lr){2-5} \cmidrule(lr){6-8}
 & \textbf{O} & \textbf{MA} & \textbf{ME} & \textbf{F} & \textbf{O} & \textbf{MA} & \textbf{ME} \\
\midrule
Baseline & 80.47 & \textbf{93.80} & 82.30 & 63.40 & 96.88 & 98.18 & 91.67 \\
CBS & 83.74 & 93.02 & 85.18 & 71.70 & 97.78 & \underline{98.43} & 95.19 \\
SMOTE & 83.31 & \underline{93.42} & 84.10 & 71.04 & 97.69 & 98.39 & 94.87 \\
ADASYN & 82.87 & 93.33 & 83.78 & 70.07 & 97.51 & 98.35 & 94.14 \\
Remix & 84.91 & 92.65 & \underline{85.60} & 75.42 & \underline{98.08} & \textbf{98.51} & 96.35 \\
Ours & \underline{85.52} & 91.61 & 85.54 & \underline{78.62} & 98.02 & 98.29 & \underline{96.93} \\
Ours + Remix & \textbf{85.84} & 90.73 & \textbf{85.61} & \textbf{80.57} & \textbf{98.15} & 98.10 & \textbf{98.38} \\
\bottomrule
\end{tabular}

\caption{Mean accuracies for different resampling and data augm

<>:1: SyntaxWarning: invalid escape sequence '\('
<>:12: SyntaxWarning: invalid escape sequence '\m'
<>:12: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\('
<>:12: SyntaxWarning: invalid escape sequence '\m'
<>:12: SyntaxWarning: invalid escape sequence '\m'
/tmp/ipykernel_4104399/4180680406.py:1: SyntaxWarning: invalid escape sequence '\('
  caption = 'Mean accuracies for different resampling and data augmentation techniques on CIFAR100-LT and CIFAR10-LT with \(\\rho = 100\), averaged over 5 runs. Best and second best results per column are in bold and underlined, respectively. \\textbf{O}: Overall, \\textbf{MA}: Many-shot classes, \\textbf{ME}: Medium-shot classes, \\textbf{F}: Few-shot classes.'
/tmp/ipykernel_4104399/4180680406.py:12: SyntaxWarning: invalid escape sequence '\m'
  latex_table = latex_table.replace('& \multicolumn{4}{r}{CIFAR100 LT} & \multicolumn{3}{r}{CIFAR10 LT} \\\\', '& \multicolumn{4}{c}{\\textbf{CIFAR100 LT}} & \mul

# losses

In [12]:
loss_path = 'comparison_loss/cifar100_0.01'

# set class groups
def get_class_groups_idx(dataset):
    spc = samples_per_class(torch.load(f'/mnt/dtafler/test/timm/vit_base_patch14_dinov2.lvd142m/{dataset}/train/labels.pt'))
    few_shot = spc <= 20
    med_shot = (100 >= spc) & (spc > 20)
    many_shot = spc > 100
    return {'few': few_shot, 'medium': med_shot, 'many': many_shot}

# rows and columns
losses = ['softmax', 'balanced_softmax', 'class_balanced_softmax', 'focal', 'equalization', 'ldam']
methods = ['plain', 'ours'] 
metrics = ['overall', 'many', 'medium', 'few']
columns = pd.MultiIndex.from_product([methods, metrics], names=['method', 'metric'])

# make dataframe
df = pd.DataFrame(index=losses, columns=columns)
df.index.name = 'loss'

for method in methods:
    
    for loss in losses:

            accs = []
            per_class_accs = []
            
            for run in range(5):
                res_path = f'{loss_path}/{method}/{loss}/run_{run}'
                accs.append(torch.load(f'{res_path}/acc.pt'))
                per_class_accs.append(torch.load(f'{res_path}/per_class.pt'))
            
            accs = torch.stack(accs, dim=0)
            per_class_accs = torch.stack(per_class_accs, dim=0)
            
            avg_acc = accs.mean(dim=0)
            avg_per_class_acc = per_class_accs.mean(dim=0)
            
            df.loc[loss, (method, 'overall')] = avg_acc.item()
            for metric in ['few', 'medium', 'many']:
                df.loc[loss, (method, metric)] = avg_per_class_acc[get_class_groups_idx('cifar100_0.01')[metric]].mean().item()
            
df = df * 100
df = df.map(lambda x: round(x, 2))
df = df.map(lambda x: f"{x:.2f}")
df.columns.names = [None, None]

df.index = ['Softmax', 'B Softmax', 'CB Softmax', 'Focal', 'Equalization', 'LDAM']

df.columns = pd.MultiIndex.from_product([['Plain', 'Plus Ours'], metrics])
df


Plain                      Plus Ours                     
             overall   many medium    few   overall   many medium    few
Softmax        80.59  93.87  82.30  63.72     85.40  91.71  85.68  77.97
B Softmax      86.26  92.26  86.38  79.35     84.09  82.83  84.02  85.58
CB Softmax     83.17  93.34  83.78  71.03     85.97  89.43  84.94  83.19
Focal          78.86  93.10  81.26  60.15     84.12  90.67  84.11  76.75
Equalization   82.41  93.83  82.04  69.92     84.52  88.76  82.19  82.30
LDAM           77.14  90.44  79.54  59.50     80.65  89.26  80.66  70.90

In [13]:
for row in df.index:
    for col in ['overall', 'many', 'medium', 'few']:
        max = df.loc[row, [('Plain', col), ('Plus Ours', col)]].max()
        df.loc[row, [('Plain', col), ('Plus Ours', col)]] = df.loc[row, [('Plain', col), ('Plus Ours', col)]].apply(lambda x: f"\\textbf{{{x}}}" if x == max else x)

df

Plain                                         \
                     overall            many          medium    few   
Softmax                80.59  \textbf{93.87}           82.30  63.72   
B Softmax     \textbf{86.26}  \textbf{92.26}  \textbf{86.38}  79.35   
CB Softmax             83.17  \textbf{93.34}           83.78  71.03   
Focal                  78.86  \textbf{93.10}           81.26  60.15   
Equalization           82.41  \textbf{93.83}           82.04  69.92   
LDAM                   77.14  \textbf{90.44}           79.54  59.50   

                   Plus Ours                                         
                     overall   many          medium             few  
Softmax       \textbf{85.40}  91.71  \textbf{85.68}  \textbf{77.97}  
B Softmax              84.09  82.83           84.02  \textbf{85.58}  
CB Softmax    \textbf{85.97}  89.43  \textbf{84.94}  \textbf{83.19}  
Focal         \textbf{84.12}  90.67  \textbf{84.11}  \textbf{76.75}  
Equalization  \textbf{84.52}  88.76  \textbf{82.19}  \textbf{82.30}  
LDAM          \textbf{80.65}  89.26  \textbf{80.66}  \textbf{70.90}

In [14]:
caption = 'Mean accuracies for different loss functions with and without CVAE data generation on CIFAR100 LT with \(\\rho = 100\), averaged over 5 runs. Best results between the two methods per column are in bold. \\textbf{O}: Overall, \\textbf{MA}: Many-shot classes, \\textbf{ME}: Medium-shot classes, \\textbf{F}: Few-shot classes. Softmax: conventional softmax categorical cross-entropy loss, B Softmax: Balanced Softmax, CB Softmax: Class Balanced Softmax.'
label = 'tab:losses100'       
 
latex_table = df.to_latex(
    column_format='lcccccccc',
    escape=False,
    multicolumn=True,
    multirow=False
)

latex_table = f"\\begin{{table}}[!ht]\n\\centering\n{latex_table}\n\\caption{{{caption}}}\n\\label{{{label}}}\n\\end{{table}}"
latex_table = latex_table.replace('& \multicolumn{4}{r}{Plain} & \multicolumn{4}{r}{Plus Ours} \\\\', '& \multicolumn{4}{c}{\\textbf{Plain}} & \multicolumn{4}{c}{\\textbf{Plus Ours}} \\\\\n\cmidrule(lr){2-5} \cmidrule(lr){6-9}')
latex_table = latex_table.replace('& overall & many & medium & few & overall & many & medium & few \\', '& \\textbf{O} & \\textbf{MA} & \\textbf{ME} & \\textbf{F} & \\textbf{O} & \\textbf{MA} & \\textbf{ME} & \\textbf{F} \\')

print(latex_table)

\begin{table}[!ht]
\centering
\begin{tabular}{lcccccccc}
\toprule
 & \multicolumn{4}{c}{\textbf{Plain}} & \multicolumn{4}{c}{\textbf{Plus Ours}} \\
\cmidrule(lr){2-5} \cmidrule(lr){6-9}
 & \textbf{O} & \textbf{MA} & \textbf{ME} & \textbf{F} & \textbf{O} & \textbf{MA} & \textbf{ME} & \textbf{F} \\
\midrule
Softmax & 80.59 & \textbf{93.87} & 82.30 & 63.72 & \textbf{85.40} & 91.71 & \textbf{85.68} & \textbf{77.97} \\
B Softmax & \textbf{86.26} & \textbf{92.26} & \textbf{86.38} & 79.35 & 84.09 & 82.83 & 84.02 & \textbf{85.58} \\
CB Softmax & 83.17 & \textbf{93.34} & 83.78 & 71.03 & \textbf{85.97} & 89.43 & \textbf{84.94} & \textbf{83.19} \\
Focal & 78.86 & \textbf{93.10} & 81.26 & 60.15 & \textbf{84.12} & 90.67 & \textbf{84.11} & \textbf{76.75} \\
Equalization & 82.41 & \textbf{93.83} & 82.04 & 69.92 & \textbf{84.52} & 88.76 & \textbf{82.19} & \textbf{82.30} \\
LDAM & 77.14 & \textbf{90.44} & 79.54 & 59.50 & \textbf{80.65} & 89.26 & \textbf{80.66} & \textbf{70.90} \\
\bottomrule
\end{tabul

<>:1: SyntaxWarning: invalid escape sequence '\('
<>:12: SyntaxWarning: invalid escape sequence '\m'
<>:12: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\('
<>:12: SyntaxWarning: invalid escape sequence '\m'
<>:12: SyntaxWarning: invalid escape sequence '\m'
/tmp/ipykernel_4104399/4232113372.py:1: SyntaxWarning: invalid escape sequence '\('
  caption = 'Mean accuracies for different loss functions with and without CVAE data generation on CIFAR100-LT with \(\\rho = 100\), averaged over 5 runs. Best results between the two methods per column are in bold. \\textbf{O}: Overall, \\textbf{MA}: Many-shot classes, \\textbf{ME}: Medium-shot classes, \\textbf{F}: Few-shot classes. Softmax: conventional softmax categorical cross-entropy loss, B Softmax: Balanced Softmax, CB Softmax: Class Balanced Softmax.'
/tmp/ipykernel_4104399/4232113372.py:12: SyntaxWarning: invalid escape sequence '\m'
  latex_table = latex_table.replace('& \multicolumn{4}{r}{Plai

In [15]:
plain = df['Plain']
ours = df['Plus Ours']

index = ['Softmax Cross Entropy', 'Balanced Softmax', 'Class Balanced Softmax', 'Focal', 'Equalization', 'LDAM']
ours.index = index
plain.index = index

ours.index = [i + '+' for i in ours.index]
# ours

combined = pd.concat([plain, ours], axis=0)
ordered_idx = ['Softmax Cross Entropy', 'Softmax Cross Entropy+', 'Balanced Softmax', 'Balanced Softmax+', 'Class Balanced Softmax', 'Class Balanced Softmax+', 'Focal', 'Focal+', 'Equalization', 'Equalization+', 'LDAM', 'LDAM+']
combined = combined.reindex(ordered_idx)

modified_index = []
for i, label in enumerate(combined.index):
    if (i % 2) == 1:  # Every second row
        modified_index.append(f"\\hspace{{7mm}}+ Ours")
    else:
        modified_index.append(label)
combined.index = modified_index

combined


,overall,many,medium,few
Softmax Cross Entropy,80.59,\textbf{93.87},82.30,63.72
\hspace{7mm}+ Ours,\textbf{85.40},91.71,\textbf{85.68},\textbf{77.97}
Balanced Softmax,\textbf{86.26},\textbf{92.26},\textbf{86.38},79.35
\hspace{7mm}+ Ours,84.09,82.83,84.02,\textbf{85.58}
Class Balanced Softmax,83.17,\textbf{93.34},83.78,71.03
\hspace{7mm}+ Ours,\textbf{85.97},89.43,\textbf{84.94},\textbf{83.19}
Focal,78.86,\textbf{93.10},81.26,60.15
\hspace{7mm}+ Ours,\textbf{84.12},90.67,\textbf{84.11},\textbf{76.75}
Equalization,82.41,\textbf{93.83},82.04,69.92
\hspace{7mm}+ Ours,\textbf{84.52},88.76,\textbf{82.19},\textbf{82.30}


In [16]:
label = 'tab:losses100'  
caption = 'Mean accuracies for different loss functions with CVAE data generation (+Ours) and without CVAE data generation on CIFAR100 LT with \(\\rho = 100\), averaged over 5 runs. Best results between the two methods are in bold. \\textbf{O}: Overall, \\textbf{MA}: Many-shot classes, \\textbf{ME}: Medium-shot classes, \\textbf{F}: Few-shot classes.'


latex_table = combined.to_latex(
    column_format='lcccc',
    escape=False,
    multicolumn=False,
    multirow=False
)

# Insert the table formatting before and after the content
latex_table = f"""\\begin{{table}}[!ht]
\\centering
\\rowcolors{{2}}{{gray!10}}{{white}}
\\begin{{tabular}}{{lcccc}}
\\toprule
 & \\textbf{{O}} & \\textbf{{MA}} & \\textbf{{ME}} & \\textbf{{F}} \\\\
\\midrule
{latex_table[latex_table.find('\\midrule') + len('\\midrule'):latex_table.rfind('\\bottomrule')].strip()}
\\bottomrule
\\end{{tabular}}
\\caption{{{caption}}}
\\label{{{label}}}
\\end{{table}}
"""

# Replace headers as needed
latex_table = latex_table.replace('& overall & many & medium & few \\', '& \\textbf{O} & \\textbf{MA} & \\textbf{ME} & \\textbf{F} \\\\')

print(latex_table)


\begin{table}[!ht]
\centering
\rowcolors{2}{gray!10}{white}
\begin{tabular}{lcccc}
\toprule
 & \textbf{O} & \textbf{MA} & \textbf{ME} & \textbf{F} \\
\midrule
Softmax Cross Entropy & 80.59 & \textbf{93.87} & 82.30 & 63.72 \\
\hspace{7mm}+ Ours & \textbf{85.40} & 91.71 & \textbf{85.68} & \textbf{77.97} \\
Balanced Softmax & \textbf{86.26} & \textbf{92.26} & \textbf{86.38} & 79.35 \\
\hspace{7mm}+ Ours & 84.09 & 82.83 & 84.02 & \textbf{85.58} \\
Class Balanced Softmax & 83.17 & \textbf{93.34} & 83.78 & 71.03 \\
\hspace{7mm}+ Ours & \textbf{85.97} & 89.43 & \textbf{84.94} & \textbf{83.19} \\
Focal & 78.86 & \textbf{93.10} & 81.26 & 60.15 \\
\hspace{7mm}+ Ours & \textbf{84.12} & 90.67 & \textbf{84.11} & \textbf{76.75} \\
Equalization & 82.41 & \textbf{93.83} & 82.04 & 69.92 \\
\hspace{7mm}+ Ours & \textbf{84.52} & 88.76 & \textbf{82.19} & \textbf{82.30} \\
LDAM & 77.14 & \textbf{90.44} & 79.54 & 59.50 \\
\hspace{7mm}+ Ours & \textbf{80.65} & 89.26 & \textbf{80.66} & \textbf{70.90} \\
\bott

<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
/tmp/ipykernel_4104399/1249376660.py:2: SyntaxWarning: invalid escape sequence '\('
  caption = 'Mean accuracies for different loss functions with CVAE data generation (+Ours) and without CVAE data generation on CIFAR100-LT with \(\\rho = 100\), averaged over 5 runs. Best results between the two methods are in bold. \\textbf{O}: Overall, \\textbf{MA}: Many-shot classes, \\textbf{ME}: Medium-shot classes, \\textbf{F}: Few-shot classes.'


## cifar10

In [17]:
loss_path = 'comparison_loss_cifar10/cifar10_0.01'

# set class groups
def get_class_groups_idx(dataset):
    spc = samples_per_class(torch.load(f'/mnt/dtafler/test/timm/vit_base_patch14_dinov2.lvd142m/{dataset}/train/labels.pt'))
    few_shot = spc <= 20
    med_shot = (100 >= spc) & (spc > 20)
    many_shot = spc > 100
    return {'few': few_shot, 'medium': med_shot, 'many': many_shot}

# rows and columns
losses = ['softmax', 'balanced_softmax', 'class_balanced_softmax', 'focal', 'equalization', 'ldam']
methods = ['plain', 'ours'] 
metrics = ['overall', 'many', 'medium', 'few']
columns = pd.MultiIndex.from_product([methods, metrics], names=['method', 'metric'])

# make dataframe
df = pd.DataFrame(index=losses, columns=columns)
df.index.name = 'loss'

for method in methods:
    
    for loss in losses:

            accs = []
            per_class_accs = []
            
            for run in range(5):
                res_path = f'{loss_path}/{method}/{loss}/run_{run}'
                accs.append(torch.load(f'{res_path}/acc.pt'))
                per_class_accs.append(torch.load(f'{res_path}/per_class.pt'))
            
            accs = torch.stack(accs, dim=0)
            per_class_accs = torch.stack(per_class_accs, dim=0)
            
            avg_acc = accs.mean(dim=0)
            avg_per_class_acc = per_class_accs.mean(dim=0)
            
            df.loc[loss, (method, 'overall')] = avg_acc.item()
            for metric in ['few', 'medium', 'many']:
                df.loc[loss, (method, metric)] = avg_per_class_acc[get_class_groups_idx('cifar10_0.01')[metric]].mean().item()
            
df = df * 100
df = df.map(lambda x: round(x, 2))
df = df.map(lambda x: f"{x:.2f}")
df.columns.names = [None, None]
df

df.index = ['Softmax', 'B Softmax', 'CB Softmax', 'Focal', 'Equalization', 'LDAM']

df.columns = pd.MultiIndex.from_product([['Plain', 'Plus Ours'], metrics])
df.drop(columns=[('Plain', 'few'), ('Plus Ours', 'few')], inplace=True)
df


Plain               Plus Ours              
             overall   many medium   overall   many medium
Softmax        96.89  98.19  91.70     98.11  98.38  97.00
B Softmax      98.26  98.42  97.63     95.74  94.80  99.47
CB Softmax     97.10  98.14  92.93     98.13  98.21  97.78
Focal          96.46  97.89  90.77     97.72  98.06  96.34
Equalization   96.90  98.15  91.92     98.09  98.34  97.06
LDAM           96.28  97.52  91.33     97.19  97.45  96.14

In [18]:
for row in df.index:
    for col in ['overall', 'many', 'medium']:
        max = df.loc[row, [('Plain', col), ('Plus Ours', col)]].max()
        df.loc[row, [('Plain', col), ('Plus Ours', col)]] = df.loc[row, [('Plain', col), ('Plus Ours', col)]].apply(lambda x: f"\\textbf{{{x}}}" if x == max else x)

df

Plain                              Plus Ours  \
                     overall            many medium         overall   
Softmax                96.89           98.19  91.70  \textbf{98.11}   
B Softmax     \textbf{98.26}  \textbf{98.42}  97.63           95.74   
CB Softmax             97.10           98.14  92.93  \textbf{98.13}   
Focal                  96.46           97.89  90.77  \textbf{97.72}   
Equalization           96.90           98.15  91.92  \textbf{98.09}   
LDAM                   96.28  \textbf{97.52}  91.33  \textbf{97.19}   

                                              
                        many          medium  
Softmax       \textbf{98.38}  \textbf{97.00}  
B Softmax              94.80  \textbf{99.47}  
CB Softmax    \textbf{98.21}  \textbf{97.78}  
Focal         \textbf{98.06}  \textbf{96.34}  
Equalization  \textbf{98.34}  \textbf{97.06}  
LDAM                   97.45  \textbf{96.14}

In [19]:
caption = 'Mean accuracies for different loss functions with and without CVAE data generation on CIFAR10 LT with \(\\rho = 100\), averaged over 5 runs. Best results between the two methods per column are in bold. \\textbf{O}: Overall, \\textbf{MA}: Many-shot classes, \\textbf{ME}: Medium-shot classes, \\textbf{F}: Few-shot classes. Softmax: conventional softmax categorical cross-entropy loss, B Softmax: Balanced Softmax, CB Softmax: Class Balanced Softmax.'
label = 'tab:losses10'       
 
latex_table = df.to_latex(
    column_format='lcccccc',
    escape=False,
    multicolumn=True,
    multirow=False
)

latex_table = f"\\begin{{table}}[!ht]\n\\centering\n{latex_table}\n\\caption{{{caption}}}\n\\label{{{label}}}\n\\end{{table}}"
latex_table = latex_table.replace('& \multicolumn{3}{r}{Plain} & \multicolumn{3}{r}{Plus Ours} \\\\', '& \multicolumn{3}{c}{\\textbf{Plain}} & \multicolumn{3}{c}{\\textbf{Plus Ours}} \\\\\n\cmidrule(lr){2-4} \cmidrule(lr){5-7}')
latex_table = latex_table.replace('& overall & many & medium & overall & many & medium \\', '& \\textbf{O} & \\textbf{MA} & \\textbf{ME} & \\textbf{O} & \\textbf{MA} & \\textbf{ME} \\')

print(latex_table)

\begin{table}[!ht]
\centering
\begin{tabular}{lcccccc}
\toprule
 & \multicolumn{3}{c}{\textbf{Plain}} & \multicolumn{3}{c}{\textbf{Plus Ours}} \\
\cmidrule(lr){2-4} \cmidrule(lr){5-7}
 & \textbf{O} & \textbf{MA} & \textbf{ME} & \textbf{O} & \textbf{MA} & \textbf{ME} \\
\midrule
Softmax & 96.89 & 98.19 & 91.70 & \textbf{98.11} & \textbf{98.38} & \textbf{97.00} \\
B Softmax & \textbf{98.26} & \textbf{98.42} & 97.63 & 95.74 & 94.80 & \textbf{99.47} \\
CB Softmax & 97.10 & 98.14 & 92.93 & \textbf{98.13} & \textbf{98.21} & \textbf{97.78} \\
Focal & 96.46 & 97.89 & 90.77 & \textbf{97.72} & \textbf{98.06} & \textbf{96.34} \\
Equalization & 96.90 & 98.15 & 91.92 & \textbf{98.09} & \textbf{98.34} & \textbf{97.06} \\
LDAM & 96.28 & \textbf{97.52} & 91.33 & \textbf{97.19} & 97.45 & \textbf{96.14} \\
\bottomrule
\end{tabular}

\caption{Mean accuracies for different loss functions with and without CVAE data generation on CIFAR10-LT with \(\rho = 100\), averaged over 5 runs. Best results between the

<>:1: SyntaxWarning: invalid escape sequence '\('
<>:12: SyntaxWarning: invalid escape sequence '\m'
<>:12: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\('
<>:12: SyntaxWarning: invalid escape sequence '\m'
<>:12: SyntaxWarning: invalid escape sequence '\m'
/tmp/ipykernel_4104399/3767370345.py:1: SyntaxWarning: invalid escape sequence '\('
  caption = 'Mean accuracies for different loss functions with and without CVAE data generation on CIFAR10-LT with \(\\rho = 100\), averaged over 5 runs. Best results between the two methods per column are in bold. \\textbf{O}: Overall, \\textbf{MA}: Many-shot classes, \\textbf{ME}: Medium-shot classes, \\textbf{F}: Few-shot classes. Softmax: conventional softmax categorical cross-entropy loss, B Softmax: Balanced Softmax, CB Softmax: Class Balanced Softmax.'
/tmp/ipykernel_4104399/3767370345.py:12: SyntaxWarning: invalid escape sequence '\m'
  latex_table = latex_table.replace('& \multicolumn{3}{r}{Plain

In [20]:
plain = df['Plain']
ours = df['Plus Ours']

index = ['Softmax Cross Entropy', 'Balanced Softmax', 'Class Balanced Softmax', 'Focal', 'Equalization', 'LDAM']
ours.index = index
plain.index = index

ours.index = [i + '+' for i in ours.index]
# ours

combined = pd.concat([plain, ours], axis=0)
ordered_idx = ['Softmax Cross Entropy', 'Softmax Cross Entropy+', 'Balanced Softmax', 'Balanced Softmax+', 'Class Balanced Softmax', 'Class Balanced Softmax+', 'Focal', 'Focal+', 'Equalization', 'Equalization+', 'LDAM', 'LDAM+']
combined = combined.reindex(ordered_idx)

modified_index = []
for i, label in enumerate(combined.index):
    if (i % 2) == 1:  # Every second row
        modified_index.append(f"\\hspace{{7mm}}+ Ours")
    else:
        modified_index.append(label)
combined.index = modified_index

combined


,overall,many,medium
Softmax Cross Entropy,96.89,98.19,91.70
\hspace{7mm}+ Ours,\textbf{98.11},\textbf{98.38},\textbf{97.00}
Balanced Softmax,\textbf{98.26},\textbf{98.42},97.63
\hspace{7mm}+ Ours,95.74,94.80,\textbf{99.47}
Class Balanced Softmax,97.10,98.14,92.93
\hspace{7mm}+ Ours,\textbf{98.13},\textbf{98.21},\textbf{97.78}
Focal,96.46,97.89,90.77
\hspace{7mm}+ Ours,\textbf{97.72},\textbf{98.06},\textbf{96.34}
Equalization,96.90,98.15,91.92
\hspace{7mm}+ Ours,\textbf{98.09},\textbf{98.34},\textbf{97.06}


In [22]:
label = 'tab:losses10'  
caption = 'Mean accuracies for different loss functions with CVAE data generation (+Ours) and without CVAE data generation on CIFAR10 LT with \(\\rho = 100\), averaged over 5 runs. Best results between the two methods are in bold. \\textbf{O}: Overall, \\textbf{MA}: Many-shot classes, \\textbf{ME}: Medium-shot classes.'


latex_table = combined.to_latex(
    column_format='lcccc',
    escape=False,
    multicolumn=False,
    multirow=False
)

# Insert the table formatting before and after the content
latex_table = f"""\\begin{{table}}[!ht]
\\centering
\\rowcolors{{2}}{{gray!10}}{{white}}
\\begin{{tabular}}{{lcccc}}
\\toprule
 & \\textbf{{O}} & \\textbf{{MA}} & \\textbf{{ME}} \\\\
\\midrule
{latex_table[latex_table.find('\\midrule') + len('\\midrule'):latex_table.rfind('\\bottomrule')].strip()}
\\bottomrule
\\end{{tabular}}
\\caption{{{caption}}}
\\label{{{label}}}
\\end{{table}}
"""

# Replace headers as needed
latex_table = latex_table.replace('& overall & many & medium \\', '& \\textbf{O} & \\textbf{MA} & \\textbf{ME} \\\\')

print(latex_table)


\begin{table}[!ht]
\centering
\rowcolors{2}{gray!10}{white}
\begin{tabular}{lcccc}
\toprule
 & \textbf{O} & \textbf{MA} & \textbf{ME} \\
\midrule
Softmax Cross Entropy & 96.89 & 98.19 & 91.70 \\
\hspace{7mm}+ Ours & \textbf{98.11} & \textbf{98.38} & \textbf{97.00} \\
Balanced Softmax & \textbf{98.26} & \textbf{98.42} & 97.63 \\
\hspace{7mm}+ Ours & 95.74 & 94.80 & \textbf{99.47} \\
Class Balanced Softmax & 97.10 & 98.14 & 92.93 \\
\hspace{7mm}+ Ours & \textbf{98.13} & \textbf{98.21} & \textbf{97.78} \\
Focal & 96.46 & 97.89 & 90.77 \\
\hspace{7mm}+ Ours & \textbf{97.72} & \textbf{98.06} & \textbf{96.34} \\
Equalization & 96.90 & 98.15 & 91.92 \\
\hspace{7mm}+ Ours & \textbf{98.09} & \textbf{98.34} & \textbf{97.06} \\
LDAM & 96.28 & \textbf{97.52} & 91.33 \\
\hspace{7mm}+ Ours & \textbf{97.19} & 97.45 & \textbf{96.14} \\
\bottomrule
\end{tabular}
\caption{Mean accuracies for different loss functions with CVAE data generation (+Ours) and without CVAE data generation on CIFAR10-LT with \(

<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
/tmp/ipykernel_4104399/552379038.py:2: SyntaxWarning: invalid escape sequence '\('
  caption = 'Mean accuracies for different loss functions with CVAE data generation (+Ours) and without CVAE data generation on CIFAR10-LT with \(\\rho = 100\), averaged over 5 runs. Best results between the two methods are in bold. \\textbf{O}: Overall, \\textbf{MA}: Many-shot classes, \\textbf{ME}: Medium-shot classes.'
